In [1]:
import pandas as pd
from glob import glob
from collections import defaultdict
import os

In [3]:
!mkdir assemblies

In [2]:
pwd

'/projects/cmi_proj/seed_grants/T1D_JaneKim/metawrap'

In [1]:
%%script bash
echo '
#!/bin/bash -l
#PBS -S /bin/bash
#PBS -e /projects/cmi_proj/seed_grants/T1D_JaneKim/logs
#PBS -o /projects/cmi_proj/seed_grants/T1D_JaneKim/logs
#PBS -l walltime=6:00:00
#PBS -l nodes=1:ppn=8
#PBS -l mem=64gb
#PBS -N t1d_metawrap_assemble

source ~/.bash_profile
conda activate metawrap-env

export TMPDIR=/panfs/panfs1.ucsd.edu/panscratch/$USER
tmp=$(mktemp -d --tmpdir)
export TMPDIR=$tmp
trap "rm -r $tmp; unset TMPDIR" EXIT
cd $tmp

name=$( sed -n "$PBS_ARRAYID"p /home/swandro/T1D/shogun/shotgun_seq_locations.tsv | cut -f1 )
r1=$( sed -n "$PBS_ARRAYID"p /home/swandro/T1D/shogun/shotgun_seq_locations.tsv | cut -f2 )
r2=$( sed -n "$PBS_ARRAYID"p /home/swandro/T1D/shogun/shotgun_seq_locations.tsv | cut -f3 )
    
out_dir=/projects/cmi_proj/seed_grants/T1D_JaneKim/metawrap/assemblies/
success_file=/projects/cmi_proj/seed_grants/T1D_JaneKim/metawrap/successful_assemblies.txt
failure_file=/projects/cmi_proj/seed_grants/T1D_JaneKim/metawrap/failed_assemblies.txt

gunzip -c $r1 > $tmp/r1.fastq
gunzip -c $r2 > $tmp/r2.fastq

metawrap assembly \
    --metaspades \
    -1 $tmp/r1.fastq \
    -2 $tmp/r2.fastq \
    -o $tmp/$name \
    -m 64 \
    -t 8 \
    -l 500
#Cleanup
#Check if completed successfully
if [ -f $tmp/$name/final_assembly.fasta ]
    then
        echo $name >> $success_file
        mv $tmp/$name $out_dir/
    #Report if failed
    else
        echo $name >> $failure_file
    fi


    
' | qsub -t 3-114%10

1321531[].barnacle.ucsd.edu
